In [1]:
import pandas as pd
import numpy as np

In [2]:
soubor = pd.read_csv('clean_data_years_filter.csv',  encoding='utf-8')
soubor.head(2)

,Období,Typ odběratele,ATC7,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet definovaných denních dávek/balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje
0,2011-01-01,LEKAR,M01AE01,125526.0,APO-IBUPROFEN 400 MG,POR TBL FLM 100X400MG,ATX,NL,D,10.0,82.83,828.3,33.3333,333.0,F
1,2011-01-01,LEKARNA,M01AE01,125526.0,APO-IBUPROFEN 400 MG,POR TBL FLM 100X400MG,ATX,NL,D,71070.0,82.83,5886728.1,33.3333,2368998.0,F


###### "clean_data_years_filter.csv" obsahuje data, kde:

###### "Typ odběratele":
'LEKARNA', 'VETERINARNI_LEKAR', 'PRODEJCE_VLP','HYGIENICKA_STANICE', 'ZDRAVOTNICKE_ZARIZENI', 'LEKAR', 'OSOBA_POSKYTUJICI_ZDRAVOTNI_PECI'

###### Abychom nic nepokazily, vytvorime novou tabulku "soubor1" a přejmenujeme poslední sloupec

In [3]:
soubor1 = soubor.loc[:,['Období', 'ATC7', 'Počet denních definovaných dávek/balení celkem']]

In [4]:
soubor1 = soubor1.rename(index=str, columns={'Počet denních definovaných dávek/balení celkem': 'DDD_total'})
soubor1.head()

,Období,ATC7,DDD_total
0,2011-01-01,M01AE01,333.0
1,2011-01-01,M01AE01,2368998.0
2,2011-01-01,M01AE01,-45533.0
3,2011-01-01,M01AE01,600.0
4,2011-01-01,M01AE01,20.0


### Jako další krok bylo třeba seskupit všechny léky obsahující stejnou molekulu ATC. 
Stejné ATC totiž může být základem pro různé léky. Použily jsme metodu groupby, pomocí které jsme vytvořily součet napříč jednotlivými ATC za každý měsíc. Protože k tomuto výpočtu z původních patnácti sloupců v tabulce nám vystačí tři, jsme do syntaxe zapsaly jen “Období”, “ATC” a “DDD_total”. 

### Zkusime spojit jednotlive ATC do vyssi urovni molekuly. 

Proto napred socteme vsechny ATC, nazev kterych se zacina s R01:

Jsou:  'R01', 'R01A', 'R01AA', 'R01AA02', 'R01AA03', 'R01AA04', 'R01AA05', 'R01AA06', 'R01AA07', 'R01AA08', 'R01AA09', 'R01AA10', 'R01AA11', 'R01AA12', 'R01AA13', 'R01AA14', 'R01AA15', 'R01AB', 'R01AB01', 'R01AB02', 'R01AB03', 'R01AB05', 'R01AB06', 'R01AB07', 'R01AB08', 'R01AD', 'R01AD01', 'R01AD02', 'R01AD03', 'R01AD04', 'R01AD05', 'R01AD06', 'R01AD07', 'R01AD08','R01AD09', 'R01AD11', 'R01AD12', 'R01AD13', 'R01AD52', 'R01AD53', 'R01AD57', 'R01AD58', 'R01AD60'








In [5]:
soubor_R01 = soubor1[soubor1['ATC7'].str.startswith('R01')]

In [6]:
soubor_R01.tail(3)

,Období,ATC7,DDD_total
87490,2018-03-01,R01AD12,60.0
87491,2018-03-01,R01AD58,113400.0
87492,2018-03-01,R01AD58,-3240.0


In [7]:
leky_R01 = soubor_R01.groupby(['Období']).agg({'Období':'first', 'DDD_total':'sum'})
leky_R01.tail()

,Období,DDD_total
Období,,
2017-11-01,2017-11-01,9.822653e+06
2017-12-01,2017-12-01,8.598804e+06
2018-01-01,2018-01-01,1.149549e+07
2018-02-01,2018-02-01,1.104067e+07
2018-03-01,2018-03-01,1.269087e+07


In [8]:
# Exportujeme
leky_R01.to_csv('leky/leky_R01.csv', encoding = 'utf-8')

In [9]:
leky_R01['Období'] = pd.to_datetime(leky_R01['Období'])

In [10]:
leky_R01['year'] = leky_R01['Období'].dt.year.astype(np.int)

In [11]:
leky_R01['month'] = leky_R01['Období'].dt.month.astype(np.int)

In [12]:
leky_R01['quater'] = leky_R01['Období'].dt.quarter.astype(np.int)
#DatetimeIndex.quarter

#### Pro statistický model jsme potřebovaly vytvořit “trend”. Náš nejstarší rok je označen nulou a každý další je o jednotku vyšší.

In [13]:
leky_R01['trend'] = leky_R01['year']-2011

In [14]:
leky_R01.tail(3)

,Období,DDD_total,year,month,quater,trend
Období,,,,,,
2018-01-01,2018-01-01,11495493.90,2018,1,1,7
2018-02-01,2018-02-01,11040668.75,2018,2,1,7
2018-03-01,2018-03-01,12690870.00,2018,3,1,7


In [15]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

C:\Users\Katerina\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Statistický model
Pro použití statistického modelu jsme potřebovaly vytvořit další sloupce pro rok, měsíc a kvartál pomocí metody knihovny pandas “.to_datetime”.

Program vnímá čísla měsíců (1–12) a kvartálů (1–4) jako hodnoty, což se nedá využít pro náš stats model. Aby program nevnímal číslo měsíců a kvartálů jako hodnotu, je třeba použít binární systém jedniček a nul.

### Proto použijeme metodu “.get_dummies” na sloupce “month” a “quater”.

In [16]:
Q01 = pd.get_dummies(leky_R01['quater'], prefix='quater')
Q01.head()

,quater_1,quater_2,quater_3,quater_4
Období,,,,
2011-01-01,1,0,0,0
2011-02-01,1,0,0,0
2011-03-01,1,0,0,0
2011-04-01,0,1,0,0
2011-05-01,0,1,0,0


In [17]:
M01 = pd.get_dummies(leky_R01['month'], prefix='month')
M01.head()

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
Období,,,,,,,,,,,,
2011-01-01,1,0,0,0,0,0,0,0,0,0,0,0
2011-02-01,0,1,0,0,0,0,0,0,0,0,0,0
2011-03-01,0,0,1,0,0,0,0,0,0,0,0,0
2011-04-01,0,0,0,1,0,0,0,0,0,0,0,0
2011-05-01,0,0,0,0,1,0,0,0,0,0,0,0


### Ted to spojime do jednoho dataframu "model_xx"
https://pandas.pydata.org/pandas-docs/stable/merging.html

In [18]:
model_R01 = pd.concat([leky_R01, M01, Q01], axis=1, join_axes=[leky_R01.index])
model_R01.head(3)

,Období,DDD_total,year,month,quater,trend,month_1,month_2,month_3,month_4,...,month_7,month_8,month_9,month_10,month_11,month_12,quater_1,quater_2,quater_3,quater_4
Období,,,,,,,,,,,,,,,,,,,,,
2011-01-01,2011-01-01,11167573.0,2011,1,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2011-02-01,2011-02-01,10160263.0,2011,2,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2011-03-01,2011-03-01,9895279.0,2011,3,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [19]:
results_R01 = smf.ols('DDD_total ~ trend + quater_1 + quater_2 + quater_3', data=model_R01).fit()
print(results_R01.summary())

                            OLS Regression Results                            
Dep. Variable:              DDD_total   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.284
Method:                 Least Squares   F-statistic:                     9.537
Date:                Sun, 20 May 2018   Prob (F-statistic):           2.21e-06
Time:                        19:30:23   Log-Likelihood:                -1377.8
No. Observations:                  87   AIC:                             2766.
Df Residuals:                      82   BIC:                             2778.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    8.43e+06   5.03e+05     16.760      0.0

In [20]:
resultsR01 = smf.ols('DDD_total ~ trend + month_1 + month_2 + month_3 + month_4 + month_5 + month_6 + month_7 + month_8 + month_9 + month_10 + month_11', data=model_R01).fit()
print(resultsR01.summary())

                            OLS Regression Results                            
Dep. Variable:              DDD_total   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     15.00
Date:                Sun, 20 May 2018   Prob (F-statistic):           2.45e-15
Time:                        19:30:23   Log-Likelihood:                -1340.8
No. Observations:                  87   AIC:                             2708.
Df Residuals:                      74   BIC:                             2740.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   7.425e+06   5.28e+05     14.052      0.0